In [6]:
import urllib3
from urllib.parse import urlencode
import bs4
from bs4 import BeautifulSoup
import mysql.connector

In [7]:
URL = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=b2AXFuhIJNJ13Rycl5fOfm3TLx3ouEbSlBwpAXEgNmZby%2FLAvMvOwdxDZCHiB2%2B34tNwOLcyMwqXA0Vs6mFvEA%3D%3D&pageNo=1&numOfRows=10000&'
DISTRICT = [11680, 11740, 11305, 11500, 11620, 11215, 11530, 11545, 11350, 11320, 11230, 11590, 11440, 11410, 11650, 11200, 11290, 11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260]

def main():

    mydb = mysql.connector.connect(
      host="localhost",
      user="sungkyun",
      password="chung741",
      database="sungkyun"
    )
    mycursor = mydb.cursor()

    getData(mydb, mycursor)


In [14]:
def getData(mydb, mycursor):
    count_limit = 0
    http = urllib3.PoolManager()
    for dist in DISTRICT[16:]:
        #for year in range(2008, 2022):
        for year in range(2006, 2022):
            print(str(dist) + ':' + str(year))
            #for month in range(12, 13):
            for month in range(1, 13):
                if month < 10:
                    ymd = str(year) + '0' + str(month)
                else:
                    ymd = str(year) + str(month)
                info = URL + 'LAWD_CD=' + str(dist) + '&DEAL_YMD=' + ymd
                resp = http.request("GET", info)
                xml_obj = bs4.BeautifulSoup(resp.data,'lxml-xml')
                insert(xml_obj, mydb, mycursor)
        count_limit = count_limit + 1
        if (count_limit >= 1000):
            print("Exceeds maximum traffic: " + count_limit)
            print("current district / year / month: " + str(dist) + " / " + str(year) + " / " + str(month))
        

In [15]:
def insert(resp, mydb, mycursor):
    rows = []
    for i in resp.findAll('item'):
        try:
            price = i.거래금액.text.replace(',', '')
            built_years = i.건축년도.text
            if int(i.월.text) < 10:
                sold_date =  i.년.text+ '-0' + i.월.text
            else:
                sold_date =  i.년.text + '-' + i.월.text
            if int(i.일.text) < 10:
                sold_date = sold_date + '-0' + i.일.text
            else:
                sold_date = sold_date + '-' + i.일.text
            street_adress = i.도로명.text
            street_adress_code = i.도로명코드.text + i.도로명건물본번호코드.text
            dong = i.법정동.text
            dong_code = i.법정동시군구코드.text + i.법정동읍면동코드.text
            name = i.아파트.text
            size = i.전용면적.text
            floor = i.층.text
            rows.append((price, built_years, sold_date, street_adress, street_adress_code, dong, dong_code, name, size, floor)) 
        except:
            print("Some attribute is missing")    
    sql = 'INSERT INTO apart_price_clone (price, built_years, sold_date, street_adress, street_adress_code, dong, dong_code, name, size, floor) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    mycursor.executemany(sql, rows)
    mydb.commit()
    print(mycursor.rowcount, "was inserted.")
    
    

In [16]:
main()

11290:2006
123 was inserted.
252 was inserted.
335 was inserted.
263 was inserted.
326 was inserted.
223 was inserted.
309 was inserted.
365 was inserted.
772 was inserted.
879 was inserted.
Some attribute is missing
639 was inserted.
347 was inserted.
11290:2007
248 was inserted.
Some attribute is missing
185 was inserted.
Some attribute is missing
266 was inserted.
178 was inserted.
140 was inserted.
186 was inserted.
251 was inserted.
226 was inserted.
Some attribute is missing
Some attribute is missing
193 was inserted.
288 was inserted.
245 was inserted.
204 was inserted.
11290:2008
280 was inserted.
307 was inserted.
490 was inserted.
435 was inserted.
445 was inserted.
Some attribute is missing
328 was inserted.
166 was inserted.
118 was inserted.
83 was inserted.
46 was inserted.
48 was inserted.
51 was inserted.
11290:2009
97 was inserted.
185 was inserted.
222 was inserted.
Some attribute is missing
258 was inserted.
Some attribute is missing
281 was inserted.
Some attribute 

Some attribute is missing
171 was inserted.
Some attribute is missing
275 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
779 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attri

267 was inserted.
201 was inserted.
155 was inserted.
213 was inserted.
247 was inserted.
110 was inserted.
5 was inserted.
5 was inserted.
5 was inserted.
5 was inserted.
5 was inserted.
5 was inserted.
11470:2006
508 was inserted.
632 was inserted.
1007 was inserted.
635 was inserted.
456 was inserted.
223 was inserted.
285 was inserted.
335 was inserted.
619 was inserted.
Some attribute is missing
Some attribute is missing
949 was inserted.
Some attribute is missing
631 was inserted.
356 was inserted.
11470:2007
158 was inserted.
124 was inserted.
166 was inserted.
171 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
122 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
187 was inserted.
Some attribute is missing
245 was inserted.
Some attribute is missing
Some attribute 

Some attribute is missing
Some attribute is missing
246 was inserted.
404 was inserted.
Some attribute is missing
Some attribute is missing
452 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
399 was inserted.
Some attribute is missing
Some attribute is missing
314 was inserted.
Some attribute is missing
Some attribute is missing
222 was inserted.
146 was inserted.
Some attribute is missing
Some attribute is missing
87 was inserted.
50 was inserted.
49 was inserted.
43 was inserted.
11560:2009
117 was inserted.
198 was inserted.
196 was inserted.
398 was inserted.
Some attribute is missing
342 was inserted.
Some attribute is missing
363 was inserted.
Some attribute is missing
405 was inserted.
Some attribute is missing
Some attribute is missing
Some attribute is missing
Some attribute is missing
466 was inserted.
Some attribute is missing
339 was inserted.
194 was inserted.
153 was inserted.
Some attribute is missing
174 was inserted.
11560:2

86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11380:2011
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11380:2012
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11380:2013
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11380:2014
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 w

86 was inserted.
86 was inserted.
11140:2017
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11140:2018
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11140:2019
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11140:2020
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
11140:2021
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was inserted.
86 was ins

In [48]:
http = urllib3.PoolManager()
resp = http.request("GET", 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=b2AXFuhIJNJ13Rycl5fOfm3TLx3ouEbSlBwpAXEgNmZby%2FLAvMvOwdxDZCHiB2%2B34tNwOLcyMwqXA0Vs6mFvEA%3D%3D&pageNo=1&numOfRows=100000&LAWD_CD=11110&DEAL_YMD=201512', headers=headers)
print(resp.data)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><\xea\xb1\xb0\xeb\x9e\x98\xea\xb8\x88\xec\x95\xa1>    82,500</\xea\xb1\xb0\xeb\x9e\x98\xea\xb8\x88\xec\x95\xa1><\xea\xb1\xb4\xec\xb6\x95\xeb\x85\x84\xeb\x8f\x84>2008</\xea\xb1\xb4\xec\xb6\x95\xeb\x85\x84\xeb\x8f\x84><\xeb\x85\x84>2015</\xeb\x85\x84><\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85>\xec\x82\xac\xec\xa7\x81\xeb\xa1\x9c8\xea\xb8\xb8</\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85><\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85\xea\xb1\xb4\xeb\xac\xbc\xeb\xb3\xb8\xeb\xb2\x88\xed\x98\xb8\xec\xbd\x94\xeb\x93\x9c>00004</\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85\xea\xb1\xb4\xeb\xac\xbc\xeb\xb3\xb8\xeb\xb2\x88\xed\x98\xb8\xec\xbd\x94\xeb\x93\x9c><\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85\xea\xb1\xb4\xeb\xac\xbc\xeb\xb6\x80\xeb\xb2\x88\xed\x98\xb8\xec\xbd\x94\xeb\x93\x9c>00000</\xeb\x8f\x84\xeb\xa1\x9c\xeb\xaa\x85\xea\xb1\xb4\xeb\xac\xbc\xeb\xb6\x80\xeb\x

In [49]:
xml_obj = bs4.BeautifulSoup(resp.data,'lxml-xml')

In [51]:
xml_obj.findAll('item')

[<item><거래금액>    82,500</거래금액><건축년도>2008</건축년도><년>2015</년><도로명>사직로8길</도로명><도로명건물본번호코드>00004</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>11110</도로명시군구코드><도로명일련번호코드>03</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4100135</도로명코드><법정동> 사직동</법정동><법정동본번코드>0009</법정동본번코드><법정동부번코드>0000</법정동부번코드><법정동시군구코드>11110</법정동시군구코드><법정동읍면동코드>11500</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>광화문풍림스페이스본(101동~105동)</아파트><월>12</월><일>10</일><일련번호>11110-2203</일련번호><전용면적>94.51</전용면적><지번>9</지번><지역코드>11110</지역코드><층>11</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>,
 <item><거래금액>    60,000</거래금액><건축년도>1981</건축년도><년>2015</년><도로명>세종대로23길</도로명><도로명건물본번호코드>00047</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>11110</도로명시군구코드><도로명일련번호코드>02</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4100190</도로명코드><법정동> 당주동</법정동><법정동본번코드>0145</법정동본번코드><법정동부번코드>0000</법정동부번코드><법정동시군구코드>11110</법정동시군구코드><법정동읍면동코드>11700</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>롯데미도파광화문빌딩</아파트><월>12</월><일>22</일><일련번호>11110-12</일련번호><전용면적>149.95</전용면적><지번>145</지번><지역코드>11

In [53]:
import pandas as pd
cols = ["price", "built_years", "sold_date", "street_adress", "dong", "name", "size", "floor"]
rows = []
  
for i in xml_obj.findAll('item'):
    price = i.거래금액.text.replace(',', '')
    built_years = i.건축년도.text
    if int(i.월.text) < 10:
        sold_date =  i.년.text+ '-0' + i.월.text
    else:
        sold_date =  i.년.text + '-' + i.월.text
    if int(i.일.text) < 10:
        sold_date = sold_date + '-0' + i.일.text
    else:
        sold_date = sold_date + '-' + i.일.text
    
    street_adress = i.도로명.text
    dong = i.법정동.text
    name = i.아파트.text
    size = i.전용면적.text
    floor = i.층.text
  
    rows.append({"price": price, "built_years": built_years, "sold_date": sold_date, "street_adress": street_adress
                 , "dong": dong, "name": name, "size": size, "floor": floor})
  
df = pd.DataFrame(rows, columns=cols)

In [54]:
df

,price,built_years,sold_date,street_adress,dong,name,size,floor
0,82500,2008,2015-12-10,사직로8길,사직동,광화문풍림스페이스본(101동~105동),94.51,11
1,60000,1981,2015-12-22,세종대로23길,당주동,롯데미도파광화문빌딩,149.95,8
2,130000,2004,2015-12-08,경희궁2길,내수동,킹스매너,194.43,6
3,105000,2004,2015-12-14,사직로8길,내수동,경희궁의아침2단지,124.17,8
4,120000,2003,2015-12-24,사직로8길,내수동,경희궁 파크팰리스,146.33,4
5,17000,2014,2015-12-17,대학로,연건동,이화에수풀,16.98,8
6,17000,2014,2015-12-18,대학로,연건동,이화에수풀,16.98,4
7,57000,2006,2015-12-29,혜화로3가길,명륜1가,렉스빌,106.98,3
8,44000,1995,2015-12-01,창경궁로,명륜2가,아남1,84.8,18
9,52000,1995,2015-12-10,창경궁로,명륜2가,아남1,84.9,12
